# Practical Examples of Data Query with AISdb

In [1]:
import os
from datetime import datetime, timedelta
import aisdb
from aisdb import DBConn, DBQuery, DomainFromPoints
import nest_asyncio
nest_asyncio.apply()

In [2]:
# Define the path to your database
dbpath='YOUR_DATABASE.db'

## Within Time Range

In [4]:
# Set the start and end times for the query
start_time = datetime.strptime("2018-01-01 00:00:00", '%Y-%m-%d %H:%M:%S')
end_time = datetime.strptime("2018-01-02 00:00:00", '%Y-%m-%d %H:%M:%S')

with aisdb.SQLiteDBConn(dbpath=dbpath) as dbconn:
    qry = aisdb.DBQuery(
        dbconn=dbconn,
        start=start_time,
        end=end_time,
        callback=aisdb.database.sqlfcn_callbacks.in_timerange_validmmsi,
    )
    rowgen = qry.gen_qry()
    
    # Convert queried rows to vessel trajectories
    tracks = aisdb.track_gen.TrackGen(rowgen, decimate=False)
    
    # Visualization
    aisdb.web_interface.visualize(
        tracks,
        # domain=domain,
        visualearth=True,
        open_browser=True,
    )

Serving HTTP assets on localhost:3000
Querying database...	done query
Opening a new browser window to display track data. Press Ctrl-C to stop the server and close the webpage
127.0.0.1:37102 - received: {'msgtype': 'meta'}
127.0.0.1:37108 - received: {'msgtype': 'validrange'}
127.0.0.1:37108 - received: {'msgtype': 'zones'}
Received KeyboardInterrupt, stopping server...


## Within geographical bounding box

In [5]:
domain = DomainFromPoints(points=[(-63.6, 44.6)], radial_distances=[50000]) # a circle with a 100km radius around the location point

with aisdb.SQLiteDBConn(dbpath=dbpath) as dbconn:
    qry = aisdb.DBQuery(
        dbconn=dbconn,
        start=start_time,
        end=end_time,
        xmin=domain.boundary['xmin'], xmax=domain.boundary['xmax'],
        ymin=domain.boundary['ymin'], ymax=domain.boundary['ymax'],
        callback=aisdb.database.sqlfcn_callbacks.in_validmmsi_bbox,
    )
    rowgen = qry.gen_qry()
    tracks = aisdb.track_gen.TrackGen(rowgen, decimate=False)
    
    aisdb.web_interface.visualize(
        tracks,
        domain=domain,
        visualearth=True,
        open_browser=True,
    )

Serving HTTP assets on localhost:3000
Querying database...	done query
Opening a new browser window to display track data. Press Ctrl-C to stop the server and close the webpage
127.0.0.1:50452 - received: {'msgtype': 'meta'}
127.0.0.1:50458 - received: {'msgtype': 'validrange'}
127.0.0.1:50458 - received: {'msgtype': 'zones'}
Received KeyboardInterrupt, stopping server...


## Combination of time range and bounding box

In [6]:
with aisdb.SQLiteDBConn(dbpath=dbpath) as dbconn:
    qry = aisdb.DBQuery(
        dbconn=dbconn,
        start=start_time,
        end=end_time,
        xmin=domain.boundary['xmin'], xmax=domain.boundary['xmax'],
        ymin=domain.boundary['ymin'], ymax=domain.boundary['ymax'],
        callback=aisdb.database.sqlfcn_callbacks.in_time_bbox_validmmsi
    )
    rowgen = qry.gen_qry()
    tracks = aisdb.track_gen.TrackGen(rowgen, decimate=False)
    
    aisdb.web_interface.visualize(
        tracks,
        domain=domain,
        visualearth=True,
        open_browser=True,
    )

Serving HTTP assets on localhost:3000
Querying database...	done query
Opening a new browser window to display track data. Press Ctrl-C to stop the server and close the webpage
127.0.0.1:58844 - received: {'msgtype': 'meta'}
127.0.0.1:58856 - received: {'msgtype': 'validrange'}
127.0.0.1:58856 - received: {'msgtype': 'zones'}
Received KeyboardInterrupt, stopping server...


## Query by listing MMSI

In [7]:
import random

def assign_colors(mmsi_list):
    colors = {}
    for mmsi in mmsi_list:
        colors[mmsi] = "#{:06x}".format(random.randint(0, 0xFFFFFF))  # Random color in hex
    return colors

# Create a function to color tracks
def color_tracks(tracks, colors):
    colored_tracks = []
    for track in tracks:
        mmsi = track['mmsi']
        color = colors.get(mmsi, "#000000")  # Default to black if no color assigned
        track['color'] = color
        colored_tracks.append(track)
    return colored_tracks

# Set the start and end times for the query
start_time = datetime.strptime("2018-01-01 00:00:00", '%Y-%m-%d %H:%M:%S')
end_time = datetime.strptime("2018-12-31 00:00:00", '%Y-%m-%d %H:%M:%S')

# Create a list of vessel MMSIs you want to track 
MMSI = [636017611,636018124,636018253]

# Assign colors to each MMSI
colors = assign_colors(MMSI)

with aisdb.SQLiteDBConn(dbpath=dbpath) as dbconn:
    qry = aisdb.DBQuery(
        dbconn=dbconn,
        start=start_time,
        end=end_time,
        mmsis = MMSI,
        callback=aisdb.database.sqlfcn_callbacks.in_timerange_inmmsi,
    )
    rowgen = qry.gen_qry()
    tracks = aisdb.track_gen.TrackGen(rowgen, decimate=False)
    
    # Color the tracks
    colored_tracks = color_tracks(tracks, colors)

    # Visualizing the tracks
    aisdb.web_interface.visualize(
        colored_tracks,
        visualearth=True,
        open_browser=True,
    )

/home/ruixin/AISdb/aisdb/database/dbqry.py:129: UserWarning: No results found in ais_201812_static
  warnings.warn(f'No results found in ais_{month}_static')


Serving HTTP assets on localhost:3000
Querying database...	done query
Opening a new browser window to display track data. Press Ctrl-C to stop the server and close the webpage
127.0.0.1:60648 - received: {'msgtype': 'meta'}
127.0.0.1:60660 - received: {'msgtype': 'validrange'}
127.0.0.1:60660 - received: {'msgtype': 'zones'}
Received KeyboardInterrupt, stopping server...
